In [ ]:
import numpy as np
import tensorflow as tf
from utils import shuffle_in_unison

def Criterion(y, y_true, lambda_coord=1, lambda_wh=1, lambda_confidence=1):
    coord_loss = tf.reduce_mean(y_true[:, :, :, 4]*tf.reduce_sum(tf.math.squared_difference(y[:, :, :, 0:2], y_true[:, :, :, 0:2]), axis=3))
    wh_loss = tf.reduce_mean(y_true[:, :, :, 4]*tf.reduce_sum(tf.math.squared_difference(y[:, :, :, 2:4], y_true[:, :, :, 2:4]), axis=3))
    confidence_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true[:, :, :, 4], logits=y[:, :, :, 4]))
    
    return lambda_coord*coord_loss + lambda_wh*wh_loss+ lambda_confidence*confidence_loss

def model(x, filt=32, use_batch_norm=True, alpha=0.1):
    Conv1 = tf.layers.conv2d(inputs=x, filters=filt, kernel_size=[5, 5], strides = 1, padding="same", activation=None, name = "Conv1")
    if use_batch_norm:
        Conv1 = tf.contrib.layers.batch_norm(inputs = Conv1)
    Conv1 = tf.maximum(Conv1, alpha*Conv1)
    Conv2 = tf.layers.conv2d(inputs=Conv1, filters=filt, kernel_size=[5, 5], strides = 2, padding="same", activation=None, name = "Conv2")
    if use_batch_norm:
        Conv2 = tf.contrib.layers.batch_norm(inputs = Conv2)
    Conv2 = tf.maximum(Conv2, alpha*Conv2)
    
    Conv3 = tf.layers.conv2d(inputs=Conv2, filters=2*filt, kernel_size=[5, 5], strides = 1, padding="same", activation=None, name = "Conv3")
    if use_batch_norm:
        Conv3 = tf.contrib.layers.batch_norm(inputs = Conv3)
    Conv3 = tf.maximum(Conv3, alpha*Conv3)
    Conv4 = tf.layers.conv2d(inputs=Conv3, filters=2*filt, kernel_size=[5, 5], strides = 2, padding="same", activation=None, name = "Conv4")
    if use_batch_norm:
        Conv4 = tf.contrib.layers.batch_norm(inputs = Conv4)
    Conv4 = tf.maximum(Conv4, alpha*Conv4)
    
    Conv5 = tf.layers.conv2d(inputs=Conv4, filters=4*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv5")
    if use_batch_norm:
        Conv5 = tf.contrib.layers.batch_norm(inputs = Conv5)
    Conv5 = tf.maximum(Conv5, alpha*Conv5)
    Conv6 = tf.layers.conv2d(inputs=Conv5, filters=4*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv6")
    if use_batch_norm:
        Conv6 = tf.contrib.layers.batch_norm(inputs = Conv6)
    Conv6 = tf.maximum(Conv6, alpha*Conv6)
    
    Conv7 = tf.layers.conv2d(inputs=Conv6, filters=8*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv7")
    if use_batch_norm:
        Conv7 = tf.contrib.layers.batch_norm(inputs = Conv7)
    Conv7 = tf.maximum(Conv7, alpha*Conv7)
    Conv8 = tf.layers.conv2d(inputs=Conv7, filters=8*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv8")
    if use_batch_norm:
        Conv8 = tf.contrib.layers.batch_norm(inputs = Conv8)
    Conv8 = tf.maximum(Conv8, alpha*Conv8)
    
    Conv9 = tf.layers.conv2d(inputs=Conv8, filters=16*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv9")
    if use_batch_norm:
        Conv9 = tf.contrib.layers.batch_norm(inputs = Conv9)
    Conv9 = tf.maximum(Conv9, alpha*Conv9)
    Conv10 = tf.layers.conv2d(inputs=Conv9, filters=16*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv10")
    if use_batch_norm:
        Conv10 = tf.contrib.layers.batch_norm(inputs = Conv10)
    Conv10 = tf.maximum(Conv10, alpha*Conv10)
    
    logits = tf.layers.conv2d(inputs=Conv10, filters=5, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv11")
    out = tf.nn.sigmoid(logits)
    return logits

images = np.load("Data/Processed/myface3_images.npy")
labels = np.load("Data/Processed/myface3_labels.npy")
validation_split = 0.8
images, labels = shuffle_in_unison(images, labels)

train_images = images[0:int(validation_split*images.shape[0])]
train_labels = labels[0:int(validation_split*labels.shape[0])]
validation_images = images[int(validation_split*images.shape[0]):]
validation_labels = labels[int(validation_split*labels.shape[0]):]

image_h = images.shape[1]
image_w = images.shape[2]
grid_h = labels.shape[1]
grid_w = labels.shape[2]

x = tf.placeholder(tf.float32, shape=[None, image_h, image_w, 3])
y_ = tf.placeholder(tf.float32, shape=[None, grid_h, grid_w, 5])

logits = model(x)
loss = Criterion(y=logits, y_true=y_)
optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(loss)

#Training parameters
logging_step = 50
epochs = 40
batch_size = 64
saver = tf.train.Saver()

with tf.Session() as sess:
    _min_ = 1000
    saver.restore(sess, "Saved models/face3")
    #sess.run(tf.global_variables_initializer())
    print("Training on %d images; Epochs: %d"%(train_images.shape[0], epochs))
    for e in range(epochs):
        print("\n Epoch %d:"%(e+1))
         #Training
        for i in range(train_images.shape[0] // batch_size):
            batch_im = train_images[i*batch_size:(i+1)*batch_size] 
            batch_l = train_labels[i*batch_size:(i+1)*batch_size]
            train_step.run(feed_dict={x: batch_im, y_: batch_l})
            if (i % logging_step == 0):
                print("Batch: %d; Loss: %f" % (i, loss.eval(feed_dict={x: batch_im, y_: batch_l})))
        validation_loss = 0        
        #Validation
        for i in range(validation_images.shape[0] // batch_size):
            batch_im = validation_images[i*batch_size:(i+1)*batch_size] 
            batch_l = validation_labels[i*batch_size:(i+1)*batch_size]
            validation_loss += loss.eval(feed_dict={x: batch_im, y_: batch_l}) / (validation_images.shape[0] // batch_size)
        print("Validation loss: ", validation_loss)
        train_images, train_labels = shuffle_in_unison(train_images, train_labels)
        if validation_loss < _min_:
            save_path = saver.save(sess, "Saved models/face3")
            _min_ = validation_loss